## Modified Slim

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from matplotlib import pyplot as plt
import deepxde as dde

######################
# ITER shape #
######################
eps = 0.32
kappa = 1.7
delta = 0.33
Amax = 1e-5

N1 = - (1 + np.arcsin(delta)) ** 2 / (eps * kappa ** 2)
N2 = (1 - np.arcsin(delta)) ** 2 / (eps * kappa ** 2)
N3 = - kappa / (eps * np.cos(np.arcsin(delta)) ** 2)

def gen_traindata(num):
    ######################
    # ITER Configuration #
    ######################

    N = num
    center = np.array([[0.0,0.0,0.0]])
    tau = np.linspace(0, 2 * np.pi, N)
    Arange = np.linspace(-Amax, Amax, N)
    # Define boundary of ellipse
    x_ellipse = np.asarray([1 + eps * np.cos(tau + np.arcsin(delta) * np.sin(tau)), 
                    eps * kappa * np.sin(tau), Arange]).T
    print(x_ellipse.shape)
    uvals = np.zeros(len(x_ellipse)).reshape(len(x_ellipse), 1)
    return x_ellipse, uvals


def pde_solovev(x, u):
    psi = u[:, 0:1]
    psi_r = dde.grad.jacobian(psi, x, i=0, j=0)
    psi_rr = dde.grad.hessian(psi, x, i=0, j=0)
    psi_zz = dde.grad.hessian(psi, x, i=1, j=1)
    A = x[:, 2:3]
    GS = psi_rr - psi_r / x[:, 0:1] + psi_zz - (1 - A) * x[:, 0:1] ** 2 - A
    return GS

def psi_r(x,u):
    return dde.grad.jacobian(u, x, i=0, j=0)
def psi_z(x,u):
    return  dde.grad.jacobian(u, x, i=0, j=1)
def psi_rr(x, u):
    return dde.grad.hessian(u, x, i=0, j=0)
def psi_zz(x, u):
    return dde.grad.hessian(u, x, i=1, j=1)

def boundary_outer(x, on_boundary):
    return on_boundary and np.isclose([x[0], x[1]], [1 + eps, 0]).all()
def boundary_inner(x, on_boundary):
    return on_boundary and np.isclose([x[0], x[1]], [1 - eps, 0]).all()
def boundary_high(x, on_boundary):
    return on_boundary and np.isclose([x[0], x[1]], [1 - delta * eps, kappa * eps]).all()

spatial_domain = dde.geometry.Ellipse_A(eps, kappa, delta, Amax=Amax) 

x, u = gen_traindata(100)

n_test = 100
x_test,u_test = gen_traindata(n_test)
x_domain = spatial_domain.random_points(n_test)
x_test = np.concatenate((x_test, x_domain))
u_test = np.concatenate((u_test, np.zeros((n_test, 1))))

bc135 = dde.PointSetBC(x,u)

data = dde.data.PDE(
    spatial_domain,
    pde_solovev,
    [bc135],
    num_domain=2048,
    num_boundary=0,
    x_test=x_test,
    y_test=u_test,
    train_distribution="LHS"
)


Using TensorFlow 2 backend.

Instructions for updating:
non-resource variables are not supported in the long term
(100, 3)
(100, 3)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/alankaptanoglu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-551ce25e64b9>", line 69, in <module>
    x_domain = spatial_domain.random_points(n_test)
  File "/Users/alankaptanoglu/deepxde_copy/deepxde/geometry/geometry_2d.py", line 198, in random_points
    if self.inside(x_new):
  File "/Users/alankaptanoglu/deepxde_copy/deepxde/geometry/geometry_2d.py", line 177, in inside
    return is_point_in_path_A(self.Amax, x[:, 0:1], x[:, 1:2], x[:, 2:3], self.x_ellipse)
  File "/Users/alankaptanoglu/deepxde_copy/deepxde/geometry/geometry_2d.py", line 768, in is_point_in_path_A
    if abs(A) == Amax:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/alankaptanoglu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.

KeyboardInterrupt: 

In [ ]:
# Plot collocation points for visual check
fig,ax=plt.subplots(1, figsize=(5,5))
ax.scatter(data.train_x_bc[:,0], data.train_x_bc[:,1], s = 2, color='r')
ax.set_title('Collocation Points')
ax.set_xlabel('R/R_0')
ax.set_ylabel(r'$u(r,z=0)$')

In [ ]:
from utils.gs_solovev_sol import GS_Linear
# tokamak = GS_Linear(A=-0.155, eps= 2/6.2, kappa=1.6, delta=0.4)

## Train Model

In [ ]:
#[0.02,0.007,0.002,0.0007,0.0002]
import os
DATE = "10122022"
CONFIG = "ITER"
LR = 2e-2
DEPTH = 2
BREADTH = 40
run = "01_100Adam_BFGS"
AF = "swish"
RUN_NAME = f"network_sweep_{DATE}_depth0{DEPTH}_breadth{BREADTH}_{AF}_lr{LR}-varying-short_lw1-10_{run}"

PATH = f"./cefron/{CONFIG}/runs/{RUN_NAME}"
# Check whether the specified path exists or not
isExist = os.path.exists(PATH)
if not isExist:
  # Create a new directory because it does not exist 
  os.makedirs(PATH)
  print("The new directory is created!")

net = dde.maps.FNN([3] + DEPTH * [BREADTH] + [1], AF, "Glorot normal")

model = dde.Model(data, net)
decay_rate = 1.0
for i in range(1):
  # Compile, train and save model
  model.compile(
      "adam", lr=LR/(decay_rate*(i+1)),
      loss_weights=[1,1]
  )
  loss_history, train_state = model.train(epochs=100, display_every = 10)
  dde.saveplot(loss_history, train_state, save_plot=True,issave=True, isplot=True,output_dir=f'./cefron/{CONFIG}/runs/{RUN_NAME}')

#### After BFGS

In [ ]:
# Compile, train and save model
model.compile(
    "L-BFGS-B",
    loss_weights=[1,100]
)
loss_history, train_state = model.train(epochs=1000, display_every = 10)
dde.saveplot(loss_history, train_state, save_plot=True,issave=True, isplot=True,output_dir=f'./cefron/{CONFIG}/runs/{RUN_NAME}')


In [ ]:
# Evaluation
from utils.utils import evaluate_A, relative_error_plot
ITER = GS_Linear(eps=eps, kappa=kappa, delta=delta)
xfull,yfull,A,psi_pred_full,psi_true_full,error=evaluate_A(ITER,model)
# X_test = spatial_domain.random_points(300)


In [ ]:
print(psi_true_full[:, :, 0], psi_true_full[:, :, -1], xfull.shape[2])
plt.figure()
plt.contour(xfull[:, :, 0], yfull[:, :, 0], psi_true_full[:, :, 0])
plt.colorbar()
plt.figure()
plt.contour(xfull[:, :, 0], yfull[:, :, 0], psi_pred_full[:, :, 0])
plt.colorbar()

In [ ]:
# Plotting Setup
zoom = ((1 + eps)-(1 - eps))*0.05
innerPoint = 1 - eps - zoom
outerPoint = 1 + eps + zoom
lowPoint   = -kappa * eps - zoom
highPoint  = kappa * eps + zoom

for i in range(0, xfull.shape[2], 5):
    print(i)
    plt.figure(i + 1, figsize=(10,10))

    levels = np.linspace(min(psi_true_full[:, :, i].reshape(-1)),0,10)
#     print(levels)
    
    plt.subplot(2, 2, 1)
    # Plot 1 - PINN Solution
    cp = plt.contour(xfull[:, :, i], yfull[:, :, i], psi_pred_full[:, :, i],levels=levels)
    # ax1.scatter(observe_x[:,0], observe_x[:,1], s = 2,c="black")
    #plt.colorbar(cp,ax=ax1).formatter.set_powerlimits((0, 0)) 
    #ax1.set_title('PINN Solution')
    #ax1.set_xlabel(r'$R/R_{0}$')
    #ax1.set_ylabel(r'$Z/R_{0}$')
    plt.grid(True)
    plt.axis(xmin=innerPoint,xmax=outerPoint,ymin=lowPoint, ymax=highPoint)

    # Plot 2 - Analytic Solution
    plt.subplot(2, 2, 2)
    cp = plt.contour(xfull[:, :, i], yfull[:, :, i], psi_true_full[:, :, i],levels=levels)
    #plt.colorbar(cp,ax=ax2).formatter.set_powerlimits((0, 0))
    #ax2.set_title('Analytical Solution')
    #ax2.set_xlabel(r'$R/R_{0}$')
    #ax2.set_ylabel(r'$Z/R_{0}$')
    plt.grid(True)
    plt.axis(xmin=innerPoint,xmax=outerPoint,ymin=lowPoint, ymax=highPoint)

    # Plot 4 - Relative Error
    plt.subplot(2, 2, 3)
    cp = plt.contourf(xfull[:, :, i], yfull[:, :, i], error[:, :, i])  #,levels=levels)
    #fig.colorbar(cp,ax=ax3).formatter.set_powerlimits((0, 0))
    #ax2.set_title('Analytical Solution')
    #ax3.set_xlabel(r'$R/R_{0}$')
    #ax3.set_ylabel(r'$Z/R_{0}$')
    plt.axis(xmin=innerPoint,xmax=outerPoint,ymin=lowPoint, ymax=highPoint)
    plt.colorbar()

    fig.tight_layout()
    #plt.savefig(f'./cefron/{CONFIG}/runs/{RUN_NAME}/analysis_after_BFGS_A' + str(A[0, 0, i]) + ' .jpg')

